# Gerardo de Miguel González

## Feature Selection Proof of Concept

### References

**::GMG::** I've used the following main references:

  - [DataCamp](https://www.datacamp.com/community/tutorials/feature-selection-python) Beginner's Guide to Feature Selection in Python. *Learn about the basics of feature selection and how to implement and investigate various feature selection techniques in Python*. Sayak Paul. September 25th, 2018.
  - [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/) Introduction to Feature Selection methods with an example (or how to select the right variables?). Saurav Kaushik, december 1, 2016.  
  
**::GMG::** I've already shown a filter method, *without success* at the moment, I have to say. Now I'm moving into *wrapper methods* (following DataCamp tutorial)
  

### Libraries

In [1]:
import pandas as pd
import numpy as np

### Dataset

**::GMG::** You may [download the data from kaggle](https://www.kaggle.com/uciml/pima-indians-diabetes-database/downloads/pima-indians-diabetes-database.zip/1) in zipped csv format (which *includes a header* with the column names by the way) using [the reference provided](https://www.kaggle.com/uciml/pima-indians-diabetes-database) in the Datacamp article *if you have an account in kaggle*. I haven't checked it but you should be able [to use the kaggle API](https://medium.com/@yvettewu.dw/tutorial-kaggle-api-google-colaboratory-1a054a382de0) too to automate the download from code *with an API key* created with your account. 

**::GMG::** I've already downloaded *manually from kaggle* (with my account) the csv dataset and placed it in a data folder.

In [2]:
#::GMG::Location of the downloaded dataset csv file
!ls data

pima-indians-diabetes.csv


In [3]:
#::GMG::Dataframe
data = pd.read_csv("data/pima-indians-diabetes.csv")

In [4]:
#::GMG::Show some samples
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
#::GMG::Show some samples
data.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [6]:
#::GMG::Get some statistics of the features and classification variable
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### Wrapper: Recursive Feature Elimination (RFE)

**::GMG::** The Recursive Feature Elimination (or RFE) works by *recursively removing attributes* and *building a model* on those attributes that remain. It uses the *model accuracy* to identify which attributes (and combination of attributes) *contribute the most* to predicting the target attribute. See [scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html). There's also a [user guide](https://scikit-learn.org/stable/modules/feature_selection.html#rfe) and two examples:

 - [RFE on pixels](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_digits.html)
 - [Recursive feature elimination with cross-validation](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html)

In [7]:
# Import your necessary dependencies
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LogisticRegression

In [8]:
# Feature extraction wrapper method from Datacamp reference
# Logistic Regression classifier as accuracy estimator to select the top 3 features
#::GMG::NOTE::Why three?
model = LogisticRegression(solver = 'liblinear')
rfe = RFE(estimator = model, 
          n_features_to_select = 3, 
          verbose = 1)

**::GMG::** I think this is best practice

In [9]:
#::GMG::Split train and test (this is not done in the datacamp tutorial)
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection
from sklearn.model_selection import train_test_split

In [10]:
#::GMG::80/20 split
train, test = train_test_split(data, test_size = 0.2, random_state=27)

In [11]:
#::GMG::Split features/target in train and test
train_features = train.iloc[:,:train.shape[1] - 1]
train_target = train['Outcome']
test_features = test.iloc[:,:test.shape[1] - 1]
test_target = test['Outcome']

In [12]:
#::GMG::Fit the RFE model and then the underlying estimator on the selectedvfeatures
rfe.fit(X = train_features, y = train_target)

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.


RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
  n_features_to_select=3, step=1, verbose=1)

In [13]:
#::GMG::no suprise in the number of selected features :)
print("Num Features: %s" % (rfe.n_features_))

Num Features: 3


In [14]:
#::GMG::Getting to kow the ropes ...
rfe.get_support()

array([ True, False, False, False, False,  True,  True, False])

In [15]:
#::GMG::Understanding how selection is made using support and ranking
print("Features: %s" % 
      [values for values in zip(data.columns, 
                                rfe.support_, 
                                rfe.ranking_)
      ]
     )

Features: [('Pregnancies', True, 1), ('Glucose', False, 2), ('BloodPressure', False, 3), ('SkinThickness', False, 5), ('Insulin', False, 6), ('BMI', True, 1), ('DiabetesPedigreeFunction', True, 1), ('Age', False, 4)]


In [16]:
#::GMG::How to get the selected features in a pythonic way ...
print("Selected Features: %s" % 
      [value for value, supported in zip(data.columns, rfe.support_) if supported]
     )

Selected Features: ['Pregnancies', 'BMI', 'DiabetesPedigreeFunction']


**::GMG::** RFE chose the top 3 features as `Pregnancies`, `BMI`, and `DiabetesPedigreeFunction`. These are *marked True* in the support array and *marked with a choice “1”* in the ranking array. This, in turn, indicates *the strength* of these features.

In [17]:
#::GMG::This is the estimator/model trained (fit) with selected features
#       May I optim hyperparameters like C, penalty, ... over CV k-folds?
rfe.estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [20]:
#::GMG::https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE.get_params
rfe.get_params()

{'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'warn',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'liblinear',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='warn',
           n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
           tol=0.0001, verbose=0, warm_start=False),
 'n_features_to_select': 3,
 'step': 1,
 'verbose': 1}

### Predict and metrics

In [21]:
#::GMG::Reduce X to the selected features and then predict using the underlying estimator
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE.predict
pred = rfe.predict(X = test_features)

In [23]:
pred.shape

(154,)

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# from sklearn.metrics import precision_score, recall_score, f1_score

In [25]:
#::GMG::Get the confusion matrix
# https://en.wikipedia.org/wiki/Confusion_matrix
# # https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix
cm_rfe = confusion_matrix(y_true = test_target, y_pred = pred)
print(cm_rfe)

[[96  9]
 [33 16]]


In [27]:
#::GMG::Flatten the confusion matrix and get its parts
# https://www.w3resource.com/numpy/manipulation/ravel.php
tn, fp, fn, tp = cm_rfe.ravel()
tn, fp, fn, tp

(96, 9, 33, 16)

**::GMG::** *Sensitivity* and *specificity* are statistical measures of the performance of a binary classification test, also known in statistics as a classification function, that are widely used in medicin. ([wikipedia](https://en.wikipedia.org/wiki/Sensitivity_and_specificity)). I can compute them from values in the confusion matrix!

In [28]:
#::GMG::specificity, selectivity or true negative rate (TNR)
# https://en.wikipedia.org/wiki/Sensitivity_and_specificity
specificity = tn / (tn + fp)
print('{:4.3f}'.format(specificity))

0.914


In [29]:
#::GMG::sensitivity, recall, hit rate, or true positive rate (TPR)
# https://en.wikipedia.org/wiki/Sensitivity_and_specificity
sensitivity = tp / (tp + fn)
print('{:4.3f}'.format(sensitivity))

0.327


In [31]:
#::GMG::The classification report shows all the metrics ...
target_names = ['healthy', 'diabetes']
print(classification_report(y_true = test_target, y_pred = pred, 
                            target_names=target_names, digits = 3))

              precision    recall  f1-score   support

     healthy      0.744     0.914     0.821       105
    diabetes      0.640     0.327     0.432        49

   micro avg      0.727     0.727     0.727       154
   macro avg      0.692     0.620     0.626       154
weighted avg      0.711     0.727     0.697       154



**::GMG::** where should I integrate feature selection in my machine learning pipeline? *Datacamp* reference author states:

> you should include the feature selection step before feeding the data to the model for training especially when you are using accuracy estimation methods such as cross-validation. This ensures that feature selection is performed on the data fold right before the model is trained. But if you perform feature selection first to prepare your data, then perform model selection and training on the selected features then it would be a blunder. If you perform feature selection on all of the data and then cross-validate, then the test data in each fold of the cross-validation procedure was also used to choose the features, and this tends to bias the performance of your machine learning model.

### Notes

In [52]:
#::GMG::Score of trained estimator over training set(?)
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE.score
print('Score (train): {:4.3f}'.format(rfe.score(train_features, train_target)))

Score (train): 0.687


In [54]:
#::GMG::Score of trained estimator over test set(?)
print('Score (test): {:4.3f}'.format(rfe.score(test_features, test_target)))

Score (test): 0.727


**::GMG::** If I want to use the (split) dataset with the selected features with other classification algorithms I can get them from `rfe` object

In [32]:
#::GMG::Perform the transformation of dataset with the selected features
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE.transform
train_features_fs = rfe.transform(X = train_features)
test_features_fs = rfe.transform(X = test_features)

In [33]:
train_features_fs.shape

(614, 3)

In [34]:
test_features_fs.shape

(154, 3)

In [35]:
#::GMG::Get the 'new' dataset pandas dataframe with the selected features
#       I'm not sure if this is necessary at all ... maybe if you're going to use it
#       further for ML task ...
train_fs = pd.DataFrame(
    data = train_features_fs, 
    columns = [value for value, supported in zip(data.columns, rfe.support_) if supported]
)

In [36]:
train_fs['Outcome'] = train_target

In [37]:
train_fs.head()

,Pregnancies,BMI,DiabetesPedigreeFunction,Outcome
0,10.0,27.7,0.182,1.0
1,2.0,21.1,0.647,0.0
2,8.0,28.4,0.259,1.0
3,0.0,59.4,2.420,0.0
4,0.0,20.0,0.236,1.0


In [38]:
train_fs.shape

(614, 4)

In [45]:
test_fs = pd.DataFrame(
    data = test_features_fs, 
    columns = [value for value, supported in zip(data.columns, rfe.support_) if supported]
)

In [46]:
test_fs['Outcome'] = test_target

In [47]:
#::GMG::Outcome yields NaN and I don't know why!!!
test_fs.head()

,Pregnancies,BMI,DiabetesPedigreeFunction,Outcome
0,7.0,28.8,0.687,NaN
1,1.0,21.8,0.416,NaN
2,8.0,32.7,0.719,NaN
3,5.0,28.8,0.153,NaN
4,5.0,41.0,0.346,NaN


In [44]:
test_fs.shape

(154, 4)

In [ ]:
#::NOTE::there's a way of estimating the number of features using cross-validation
#        see https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html
